In [3]:

!/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/bin/python -m pip install numpy pandas pygam


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 11.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.6/29.6 MB 3.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 3.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/bin/python -m pip install --upgrade pip


In [3]:
import numpy as np
import pandas as pd
from pygam import LogisticGAM, LinearGAM, s
from calibrate_nuisances import estimate_cdml_ate, bootstrap_cdml_ate

# Generate simulated data
np.random.seed(42)
n = 1000  # Number of observations

# Covariates: Uniform distribution between -1 and 1
X = np.random.uniform(-1, 1, size=(n, 3))

# Logistic treatment assignment based on covariates
logit_p = 1 / (1 + np.exp(-np.dot(X, np.array([0.5, -0.3, 0.2]))))
A = np.random.binomial(1, logit_p)

# Outcome generation
Y = 1 + 2 * A + 0.5 * np.random.normal(size=n)  # Simple linear treatment effect with noise

# Train outcome model using GAM (separately for control and treatment groups)
outcome_model_control = LinearGAM(s(0) + s(1) + s(2)).fit(X[A == 0], Y[A == 0])
outcome_model_treatment = LinearGAM(s(0) + s(1) + s(2)).fit(X[A == 1], Y[A == 1])

# Predicted outcome matrix
mu_mat = np.column_stack((
    outcome_model_control.predict(X),
    outcome_model_treatment.predict(X)
))

# Train propensity model using GAM
propensity_model = LogisticGAM(s(0) + s(1) + s(2)).fit(X, A)

# Propensity score matrix
pi_mat = np.column_stack((
    1 - propensity_model.predict_proba(X),  # Probability of control
    propensity_model.predict_proba(X)       # Probability of treatment
))

# Apply the estimator
ate_estimates = estimate_cdml_ate(A, Y, mu_mat, pi_mat)
print("Estimated ATE using ICDML:")
print(ate_estimates)

# Apply the bootstrap estimator
ate_bootstrap_results = bootstrap_cdml_ate(A, Y, mu_mat, pi_mat)
print("\nBootstrap ATE Results with Confidence Intervals:")
print(ate_bootstrap_results)


ModuleNotFoundError: No module named 'numpy'